# Market Predictor

## Loading data

In [8]:
import numpy as np
import pandas as pd


df = pd.read_csv('csv/000997.csv')
df.head()

,Date,Open,Close,ChgVol,Chg,Low,High,Vol,Amt,TORate
0,2017-02-20,17.83,18.00,0.16,0.90,17.70,18.01,65991,11806.87,0.71
1,2017-02-17,18.10,17.84,-0.26,-1.44,17.80,18.17,95004,17051.83,1.03
2,2017-02-16,18.28,18.10,0.11,0.61,18.01,18.28,60777,10991.66,0.66
3,2017-02-15,18.30,17.99,-0.31,-1.69,17.93,18.37,102305,18599.72,1.11
4,2017-02-14,18.25,18.30,0.04,0.22,18.20,18.36,63664,11634.16,0.69


## Cleaning the data

### Calculating amplitude

In [9]:
df['Amp'] = df.apply(lambda row: (row['High'] - row['Low']) / row['Open'] * 100 ,axis=1)

### Droping redundant columns

In [10]:
clean_df = df.iloc[::-1].loc[:,['Chg', 'TORate','Amp']][1:]
clean_df[0:].reset_index(drop=True).head()

,Chg,TORate,Amp
0,0.40,7.47,2.840909
1,3.08,8.97,5.472496
2,-0.16,6.69,3.217889
3,-0.71,3.80,1.107726
4,-1.66,3.25,1.722222


### Creating dataframe of training samples

In [11]:
df_list = []

for i in range(10):
    df_list.append(clean_df[i:].reset_index(drop=True))

df_list.append(clean_df[['Chg']][10:].reset_index(drop=True))

sample_df = pd.concat(df_list, axis=1, join='inner', ignore_index=True)
sample_df.head()

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,0.40,7.47,2.840909,3.08,8.97,5.472496,-0.16,6.69,3.217889,-0.71,...,-0.86,1.26,2.082755,-0.31,1.72,1.098592,-1.15,2.51,1.380282,-2.22
1,3.08,8.97,5.472496,-0.16,6.69,3.217889,-0.71,3.80,1.107726,-1.66,...,-0.31,1.72,1.098592,-1.15,2.51,1.380282,-2.22,2.07,2.724177,-2.04
2,-0.16,6.69,3.217889,-0.71,3.80,1.107726,-1.66,3.25,1.722222,2.05,...,-1.15,2.51,1.380282,-2.22,2.07,2.724177,-2.04,3.58,4.903678,5.34
3,-0.71,3.80,1.107726,-1.66,3.25,1.722222,2.05,2.09,2.760563,-0.77,...,-2.22,2.07,2.724177,-2.04,3.58,4.903678,5.34,2.38,8.985765,2.23
4,-1.66,3.25,1.722222,2.05,2.09,2.760563,-0.77,1.67,1.405733,-0.86,...,-2.04,3.58,4.903678,5.34,2.38,8.985765,2.23,4.57,3.954802,-0.11


## Building the deep neuron network

### creating the training sets

In [12]:
trainX = sample_df.loc[:, 0:29].values
trainX = np.array(trainX, ndmin=2)
trainX.shape

(3789, 30)

In [13]:
trainY = sample_df[30].values
trainY = np.array(trainY, ndmin=2).T
trainY.shape

(3789, 1)

### Building Network

In [17]:
import tensorflow as tf
import tflearn

tf.reset_default_graph()
tflearn.init_graph(num_cores=8, gpu_memory_fraction=0.5)

net = tflearn.input_data(shape=[None, trainX.shape[1]])
net = tflearn.fully_connected(net, 15, activation='relu')
#net = tflearn.dropout(net, 0.5)
net = tflearn.fully_connected(net, 1)
net = tflearn.regression(net, optimizer='sgd', learning_rate=0.1, loss=tflearn.objectives.mean_square)

model = tflearn.DNN(net)

### Training Network

In [16]:
model.fit(trainX, trainY, validation_set=0.1, show_metric=True, shuffle=True, batch_size=None, n_epoch=1000)

Training Step: 1079  | total loss: 10.14788 | time: 0.728s
| SGD | epoch: 020 | loss: 10.14788 - binary_acc: 0.0142 -- iter: 3392/3410
Training Step: 1080  | total loss: 9.87487 | time: 1.743s
| SGD | epoch: 020 | loss: 9.87487 - binary_acc: 0.0127 | val_loss: 9.32460 - val_acc: 0.0079 -- iter: 3410/3410
--
